In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import UnstructuredPDFLoader
# from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from coexpert.params import OPENAI_API_KEY


In [2]:
path = './raw_data'
loader = PyPDFDirectoryLoader(path)
documents = loader.load()
print(documents)

# split the documents in small chunks
# Change the chunk_size and chunk_overlap as needed
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)

pages = all_splits

# Replace book.pdf with any pdf of your choice
# loader = UnstructuredPDFLoader("../../raw_data/_51113879-001A_NG4_Tachy_CRT-D_PTM_en_S.pdf")
# pages = loader.load_and_split()


[Document(page_content='REFERENCE GUIDE\nRESONATE™HFICD,\nRESONATE™ELICD,\nPERCIVA™HFICD,\nPERCIVA™ICD,\nVIGILANT™ELICD,\nMOMENTUM™ELICD\nIMPLANTABLE CARDIO VERTER DEFIBRILLATOR\nRREEFFD520,D521,D532,D533,D420,D421,D432,D433,D500,D501,D512,D513,\nD400,D401,D412,D413,D220,D221,D232,D233,D120,D121\nCAUTION: Federallaw(USA)\nrestricts thisdevicetosalebyoron\ntheorderofaphysician trainedor\nexperience dindeviceimplant and\nfollow-up procedures.', metadata={'source': 'raw_data/51114088-001A_NG4 Tachy MRI ICD_RG_en_S.pdf', 'page': 0}), Document(page_content='', metadata={'source': 'raw_data/51114088-001A_NG4 Tachy MRI ICD_RG_en_S.pdf', 'page': 1}), Document(page_content='ABOUTTHISMANUAL\nThisliteratureisintended forusebyprofessionals trainedorexperienc edindeviceimplant and/orfollow-up\nprocedures.\nThesefamilies ofimplantable cardioverter defibrillators (ICDs)containbothsingle-anddual-chamber pulse\ngeneratorsthatprovideventricular tachyarrhythmia therapy,bradycardia pacing,andavarietyofdia

In [3]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
docsearch = Chroma.from_documents(pages, embeddings).as_retriever()


Using embedded DuckDB without persistence: data will be transient


In [25]:
query = "Quelle est la différence entre le seuil automatique et la capture automatique pour le VD sur ICD?"
docs = docsearch.get_relevant_documents(query)
# chain = load_qa_chain(OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY), chain_type="stuff")

chain = load_qa_chain(ChatOpenAI(
    temperature=0, openai_api_key=OPENAI_API_KEY), chain_type="stuff")
output = chain.run(input_documents=docs, question=query)
print(output)


Les critères de basculement de AAI en DDD du mode Rythmiq sont les suivants:
- Détection d'une perte de synchronisation AV
- Enregistrement d'un épisode Rythmiq avec des données électrogrammes
- Activation du mode DDD(R) pendant la phase Rythmiq
- Activation de la fonction ATR si elle est programmée en mode DDD(R)
- Activation de la fonction de lissage de fréquence si elle est programmée en mode DDD(R)
